In [3]:
#pip install nba_api

In [2]:
import pandas as pd

In [2]:
import pandas as pd
from nba_api.stats.endpoints import leaguegamefinder, boxscoretraditionalv2
from nba_api.stats.static import teams
import time
import json

def get_nba_teams():
    """Get all NBA team information"""
    return teams.get_teams()

def get_season_games(season='2024-25'):
    """Get all games for the 2024-25 season"""
    print(f"Fetching games for {season} season...")
    
    # Get games using LeagueGameFinder
    gamefinder = leaguegamefinder.LeagueGameFinder(
        season_nullable=season,
        season_type_nullable='Regular Season'
    )
    
    games_df = gamefinder.get_data_frames()[0]
    
    # Remove duplicate games (each game appears twice, once for each team)
    games_df = games_df.drop_duplicates(subset=['GAME_ID'], keep='first')
    
    return games_df

def get_box_score(game_id):
    """Get detailed box score for a specific game"""
    try:
        print(f"Fetching box score for game {game_id}...")
        
        boxscore = boxscoretraditionalv2.BoxScoreTraditionalV2(game_id=game_id)
        
        # Get player stats
        player_stats = boxscore.get_data_frames()[0]
        
        # Get team stats
        team_stats = boxscore.get_data_frames()[1]
        
        time.sleep(1)  # Be respectful to the API
        
        return {
            'player_stats': player_stats,
            'team_stats': team_stats
        }
    except Exception as e:
        print(f"Error fetching box score for game {game_id}: {e}")
        return None

def scrape_nba_season_2024_25():
    """Main function to scrape NBA 2024-25 season data"""
    all_data = {
        'games': [],
        'box_scores': {}
    }
    
    try:
        # Get all teams
        nba_teams = get_nba_teams()
        print(f"Found {len(nba_teams)} NBA teams")
        
        # Get season games
        games_df = get_season_games('2024-25')
        print(f"Found {len(games_df)} games")
        
        # Convert games to list of dictionaries
        games_list = games_df.to_dict('records')
        all_data['games'] = games_list
        
        print(f"\nSample game data:")
        if games_list:
            sample_game = games_list[0]
            for key, value in sample_game.items():
                print(f"{key}: {value}")
        
        # Get box scores for first 10 games (you can modify this)
        #print(f"\nFetching box scores for first 10 games...")
        #for i, game in enumerate(games_list[:10]):
            #game_id = game['GAME_ID']

        # Get box scores for all games (you can modify this)
        print(f"\nFetching box scores for all games...")
        for i, game in enumerate(games_list[:301]):
            game_id = game['GAME_ID']
            
            box_score_data = get_box_score(game_id)
            if box_score_data:
                all_data['box_scores'][game_id] = {
                    'player_stats': box_score_data['player_stats'].to_dict('records'),
                    'team_stats': box_score_data['team_stats'].to_dict('records')
                }
            
            print(f"Processed {i+1}/1758 games")
        
        return all_data
        
    except Exception as e:
        print(f"Error in main scraping function: {e}")
        return all_data

def save_data_to_files(data):
    """Save the scraped data to JSON and CSV files"""
    
    # Save complete data as JSON
    with open('nba_2024_25_complete_data11.json', 'w') as f:
        json.dump(data, f, indent=2, default=str)
    
    # Save games data as CSV
    if data['games']:
        games_df = pd.DataFrame(data['games'])
        games_df.to_csv('nba_2024_25_games11.csv', index=False)
        print(f"Saved {len(games_df)} games to nba_2024_25_games11.csv")
    
    # Save box score data
    if data['box_scores']:
        all_player_stats = []
        all_team_stats = []
        
        for game_id, box_score in data['box_scores'].items():
            # Add game_id to each record
            for player in box_score['player_stats']:
                player['GAME_ID'] = game_id
                all_player_stats.append(player)
            
            for team in box_score['team_stats']:
                team['GAME_ID'] = game_id
                all_team_stats.append(team)
        
        if all_player_stats:
            player_df = pd.DataFrame(all_player_stats)
            player_df.to_csv('nba_2024_25_player_stats11.csv', index=False)
            print(f"Saved {len(player_df)} player stat records to nba_2024_25_player_stats11.csv")
        
        if all_team_stats:
            team_df = pd.DataFrame(all_team_stats)
            team_df.to_csv('nba_2024_25_team_stats11.csv', index=False)
            print(f"Saved {len(team_df)} team stat records to nba_2024_25_team_stats11.csv")

if __name__ == "__main__":
    print("Starting NBA 2024-25 season data scraping using NBA API...")
    
    # Install required package first:
    # pip install nba_api pandas
    
    # Scrape the data
    scraped_data = scrape_nba_season_2024_25()
    
    # Save to files
    save_data_to_files(scraped_data)
    
    print("\nScraping completed!")
    print(f"Total games found: {len(scraped_data['games'])}")
    print(f"Box scores collected: {len(scraped_data['box_scores'])}")


In [2]:
import pandas as pd
import glob

# If the folder is in the same directory as your notebook:
path = "Team Stats Files"

# Get all CSV files in that folder
files = glob.glob(path + "/*.csv")

# Combine them
dfs = []
for file in files:
    df = pd.read_csv(file)
    dfs.append(df)

combined_df = pd.concat(dfs, ignore_index=True)

# Save combined file to same folder (or wherever you want)
combined_df.to_csv("combined_file.csv", index=False)

print("Done! All CSV files combined.")

Done! All CSV files combined.


In [4]:
games_df = pd.read_csv('nba_2024_25_games.csv')
team_stats_df = pd.read_csv('combined_file.csv')

# ----------------------------
# Define official NBA team abbreviations
# ----------------------------
# Update this if needed
nba_teams = [
    'ATL', 'BOS', 'BKN', 'CHA', 'CHI', 'CLE', 'DAL', 'DEN',
    'DET', 'GSW', 'HOU', 'IND', 'LAC', 'LAL', 'MEM', 'MIA',
    'MIL', 'MIN', 'NOP', 'NYK', 'OKC', 'ORL', 'PHI', 'PHX',
    'POR', 'SAC', 'SAS', 'TOR', 'UTA', 'WAS'
]

# ----------------------------
# Filter: Keep only rows where TEAM_ABBREVIATION is valid
# ----------------------------

# For games data
games_clean = games_df[
    games_df['TEAM_ABBREVIATION'].isin(nba_teams)
].copy()

# For team stats data
team_stats_clean = team_stats_df[
    team_stats_df['TEAM_ABBREVIATION'].isin(nba_teams)
].copy()

# ----------------------------
# Save the cleaned data to new Excel files
# ----------------------------
games_clean.to_csv('games_cleaned.csv', index=False)
team_stats_clean.to_csv('team_stats_cleaned.csv', index=False)

print("✅ Data cleaned! Files saved: games_cleaned.csv & team_stats_cleaned.csv")

✅ Data cleaned! Files saved: games_cleaned.csv & team_stats_cleaned.csv


In [3]:
# Load your CSV file
df = pd.read_csv("games_cleaned.csv")

# Select only the columns you care about
team_map = df[['TEAM_ID', 'TEAM_ABBREVIATION']]

# Drop duplicate rows — keep unique mappings only
unique_map = team_map.drop_duplicates()

unique_map.head(30)

,TEAM_ID,TEAM_ABBREVIATION
0,1610612741,CHI
1,1610612762,UTA
2,1610612746,LAC
3,1610612756,PHX
4,1610612766,CHA
5,1610612760,OKC
6,1610612757,POR
7,1610612754,IND
8,1610612761,TOR
9,1610612765,DET


In [10]:
def process_nba_data(teams_csv_path, games_csv_path, output_csv_path):
    """
    Process NBA teams and games data to map home/away team information
    
    Args:
        teams_csv_path: Path to the NBA teams CSV file
        games_csv_path: Path to the games CSV file  
        output_csv_path: Path for the output CSV file
    """
    
    # Read the CSV files
    teams_df = pd.read_csv(teams_csv_path)
    games_df = pd.read_csv(games_csv_path)
    
    # Create a mapping dictionary from team abbreviation to team info
    # Assuming the teams file has columns like 'TEAM_ABBREVIATION', 'TEAM_ID', 'TEAM_NAME'
    team_mapping = {}
    for _, row in teams_df.iterrows():
        abbrev = row['TEAM_ABBREVIATION']
        team_mapping[abbrev] = {
            'id': row['TEAM_ID'],
            'name': row['TEAM_NAME'],
            'abbreviation': abbrev
        }
    
    # Function to parse matchup and extract team info
    def parse_matchup(matchup):
        """Parse matchup string like 'CHI @ PHI' to extract away and home teams"""
        if ' @ ' in matchup:
            away_abbrev, home_abbrev = matchup.split(' @ ')
            return away_abbrev.strip(), home_abbrev.strip()
        elif ' vs. ' in matchup:
            home_abbrev, away_abbrev = matchup.split(' vs. ')
            return away_abbrev.strip(), home_abbrev.strip()
        return None, None
    
    # Create new columns for home and away team information
    games_df['AWAY_TEAM_ABBREV'] = ''
    games_df['HOME_TEAM_ABBREV'] = ''
    games_df['AWAY_TEAM_ID'] = ''
    games_df['HOME_TEAM_ID'] = ''
    games_df['AWAY_TEAM_NAME'] = ''
    games_df['HOME_TEAM_NAME'] = ''
    
    # Process each game
    for idx, row in games_df.iterrows():
        matchup = row['MATCHUP']  # Assuming the column name is 'matchup'
        away_abbrev, home_abbrev = parse_matchup(matchup)
        
        if away_abbrev and home_abbrev:
            # Map away team
            if away_abbrev in team_mapping:
                games_df.at[idx, 'AWAY_TEAM_ABBREV'] = away_abbrev
                games_df.at[idx, 'AWAY_TEAM_ID'] = team_mapping[away_abbrev]['id']
                games_df.at[idx, 'AWAY_TEAM_NAME'] = team_mapping[away_abbrev]['name']
            else:
                print(f"Warning: Away team abbreviation '{away_abbrev}' not found in teams data")
            
            # Map home team
            if home_abbrev in team_mapping:
                games_df.at[idx, 'HOME_TEAM_ABBREV'] = home_abbrev
                games_df.at[idx, 'HOME_TEAM_ID'] = team_mapping[home_abbrev]['id']
                games_df.at[idx, 'HOME_TEAM_NAME'] = team_mapping[home_abbrev]['name']
            else:
                print(f"Warning: Home team abbreviation '{home_abbrev}' not found in teams data")
    
    # Handle column name conflicts by prefixing original columns if they exist
    # This preserves the original data while adding the new mapped columns
    original_columns = ['TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME']
    for col in original_columns:
        if col in games_df.columns:
            games_df.rename(columns={col: f'ORIGINAL_{col}'}, inplace=True)
    
    # Save the processed data
    games_df.to_csv(output_csv_path, index=False)
    print(f"Processed data saved to {output_csv_path}")
    
    # Print summary
    print("\nSummary:")
    print(f"Total games processed: {len(games_df)}")
    print(f"Games with valid home/away mapping: {len(games_df[games_df['HOME_TEAM_ID'] != ''])}")
    
    return games_df


# Example usage
if __name__ == "__main__":
    # Replace these paths with your actual file paths
    teams_file = "nba_teams.csv"
    games_file = "games_cleaned.csv"
    output_file = "nba_games_with_team_mapping.csv"
    
    # Process the data
    result_df = process_nba_data(teams_file, games_file, output_file)
    
    # Display first few rows to verify
    print("\nFirst 5 rows of processed data:")
    print(result_df[['MATCHUP', 'AWAY_TEAM_ABBREV', 'AWAY_TEAM_NAME', 'HOME_TEAM_ABBREV', 'HOME_TEAM_NAME']].head())
    

Processed data saved to nba_games_with_team_mapping.csv

Summary:
Total games processed: 1230
Games with valid home/away mapping: 1230

First 5 rows of processed data:
     MATCHUP AWAY_TEAM_ABBREV        AWAY_TEAM_NAME HOME_TEAM_ABBREV  \
0  CHI @ PHI              CHI         Chicago Bulls              PHI   
1  UTA @ MIN              UTA             Utah Jazz              MIN   
2  LAC @ GSW              LAC  Los Angeles Clippers              GSW   
3  PHX @ SAC              PHX          Phoenix Suns              SAC   
4  CHA @ BOS              CHA     Charlotte Hornets              BOS   

           HOME_TEAM_NAME  
0      Philadelphia 76ers  
1  Minnesota Timberwolves  
2   Golden State Warriors  
3        Sacramento Kings  
4          Boston Celtics  


In [11]:
teams_df = pd.read_csv('nba_games_with_team_mapping.csv')
teams_df.head(10)

,SEASON_ID,ORIGINAL_TEAM_ID,ORIGINAL_TEAM_ABBREVIATION,ORIGINAL_TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,BLK,TOV,PF,PLUS_MINUS,AWAY_TEAM_ABBREV,HOME_TEAM_ABBREV,AWAY_TEAM_ID,HOME_TEAM_ID,AWAY_TEAM_NAME,HOME_TEAM_NAME
0,22024,1610612741,CHI,Chicago Bulls,22401191.0,2025-04-13,CHI @ PHI,W,240,122,...,4,12,17,20.0,CHI,PHI,1610612741,1610612755,Chicago Bulls,Philadelphia 76ers
1,22024,1610612762,UTA,Utah Jazz,22401195.0,2025-04-13,UTA @ MIN,L,241,105,...,5,13,15,-11.0,UTA,MIN,1610612762,1610612750,Utah Jazz,Minnesota Timberwolves
2,22024,1610612746,LAC,LA Clippers,22401198.0,2025-04-13,LAC @ GSW,W,265,124,...,3,16,21,5.0,LAC,GSW,1610612746,1610612744,Los Angeles Clippers,Golden State Warriors
3,22024,1610612756,PHX,Phoenix Suns,22401200.0,2025-04-13,PHX @ SAC,L,241,98,...,1,11,9,-11.0,PHX,SAC,1610612756,1610612758,Phoenix Suns,Sacramento Kings
4,22024,1610612766,CHA,Charlotte Hornets,22401187.0,2025-04-13,CHA @ BOS,L,240,86,...,4,17,16,-7.0,CHA,BOS,1610612766,1610612738,Charlotte Hornets,Boston Celtics
5,22024,1610612760,OKC,Oklahoma City Thunder,22401196.0,2025-04-13,OKC @ NOP,W,241,115,...,4,10,25,15.0,OKC,NOP,1610612760,1610612740,Oklahoma City Thunder,New Orleans Pelicans
6,22024,1610612757,POR,Portland Trail Blazers,22401199.0,2025-04-13,POR vs. LAL,W,240,109,...,6,14,16,28.0,LAL,POR,1610612747,1610612757,Los Angeles Lakers,Portland Trail Blazers
7,22024,1610612754,IND,Indiana Pacers,22401189.0,2025-04-13,IND @ CLE,W,289,126,...,10,13,20,8.0,IND,CLE,1610612754,1610612739,Indiana Pacers,Cleveland Cavaliers
8,22024,1610612761,TOR,Toronto Raptors,22401197.0,2025-04-13,TOR @ SAS,L,240,118,...,1,13,23,-7.0,TOR,SAS,1610612761,1610612759,Toronto Raptors,San Antonio Spurs
9,22024,1610612765,DET,Detroit Pistons,22401192.0,2025-04-13,DET @ MIL,L,264,133,...,3,13,19,-7.0,DET,MIL,1610612765,1610612749,Detroit Pistons,Milwaukee Bucks


In [12]:
df = pd.read_csv('nba_teams.csv')
df.head(30)

,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,CONFERENCE,DIVISION
0,1610612741,Chicago Bulls,CHI,East,Central
1,1610612762,Utah Jazz,UTA,West,Northwest
2,1610612746,Los Angeles Clippers,LAC,West,Pacific
3,1610612756,Phoenix Suns,PHX,West,Pacific
4,1610612766,Charlotte Hornets,CHA,East,Southeast
5,1610612760,Oklahoma City Thunder,OKC,West,Northwest
6,1610612757,Portland Trail Blazers,POR,West,Northwest
7,1610612754,Indiana Pacers,IND,East,Central
8,1610612761,Toronto Raptors,TOR,East,Atlantic
9,1610612765,Detroit Pistons,DET,East,Central


In [13]:
df = pd.read_csv('team_stats_cleaned.csv')
df.head(30)

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CITY,MIN,FGM,FGA,FG_PCT,FG3M,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,22401160,1610612748,Heat,MIA,Miami,240:00,44,92,0.478,12,...,11,28,39,28,11,3,13,10,111,-8.0
1,22401160,1610612741,Bulls,CHI,Chicago,240:00,46,89,0.517,14,...,10,36,46,33,8,7,16,14,119,8.0
2,22401162,1610612762,Jazz,UTA,Utah,265:00,50,98,0.510,15,...,16,40,56,27,14,4,24,26,133,7.0
3,22401162,1610612757,Trail Blazers,POR,Portland,265:00,48,104,0.462,11,...,14,29,43,27,13,5,16,23,126,-7.0
4,22401164,1610612756,Suns,PHX,Phoenix,240:00,42,87,0.483,15,...,9,29,38,31,10,3,13,17,112,-13.0
5,22401164,1610612760,Thunder,OKC,Oklahoma City,240:00,45,86,0.523,12,...,8,36,44,24,11,2,10,18,125,13.0
6,22401158,1610612761,Raptors,TOR,Toronto,240:00,49,104,0.471,19,...,16,39,55,37,8,2,8,16,126,30.0
7,22401158,1610612766,Hornets,CHA,Charlotte,240:00,36,82,0.439,11,...,5,36,41,23,5,3,19,13,96,-30.0
8,22401165,1610612758,Kings,SAC,Sacramento,240:00,43,92,0.467,16,...,11,29,40,25,2,5,8,22,116,-8.0
9,22401165,1610612743,Nuggets,DEN,Denver,240:00,43,86,0.500,13,...,12,33,45,30,7,5,7,16,124,8.0


In [14]:
import pandas as pd

def add_home_away_info(team_stats_path, games_mapping_path, output_path):
    """
    Add IS_HOME, OPPONENT_ID, and STAT_ID fields to team stats file
    """
    # Read both CSV files
    team_stats_df = pd.read_csv(team_stats_path)
    games_df = pd.read_csv(games_mapping_path)
    
    # Create a mapping from GAME_ID to home/away team info
    game_mapping = {}
    for _, row in games_df.iterrows():
        game_id = row['GAME_ID']
        game_mapping[game_id] = {
            'home_team_id': row['HOME_TEAM_ID'],
            'away_team_id': row['AWAY_TEAM_ID']
        }
    
    # Add new columns
    team_stats_df['IS_HOME'] = False
    team_stats_df['OPPONENT_ID'] = ''
    team_stats_df['STAT_ID'] = ''
    
    # Process each row in team stats
    for idx, row in team_stats_df.iterrows():
        game_id = row['GAME_ID']
        team_id = row['TEAM_ID']
        
        if game_id in game_mapping:
            home_team_id = game_mapping[game_id]['home_team_id']
            away_team_id = game_mapping[game_id]['away_team_id']
            
            # Determine if this team is home or away
            if team_id == home_team_id:
                team_stats_df.at[idx, 'IS_HOME'] = True
                team_stats_df.at[idx, 'OPPONENT_ID'] = away_team_id
            elif team_id == away_team_id:
                team_stats_df.at[idx, 'IS_HOME'] = False
                team_stats_df.at[idx, 'OPPONENT_ID'] = home_team_id
            else:
                print(f"Warning: Team ID {team_id} not found in game {game_id}")
        
        # Create STAT_ID as concatenation of GAME_ID + TEAM_ID
        team_stats_df.at[idx, 'STAT_ID'] = f"{game_id}{team_id}"
    
    # Save the updated file
    team_stats_df.to_csv(output_path, index=False)
    print(f"Updated team stats saved to {output_path}")
    
    # Print summary
    home_games = len(team_stats_df[team_stats_df['IS_HOME'] == True])
    away_games = len(team_stats_df[team_stats_df['IS_HOME'] == False])
    print(f"Home games: {home_games}, Away games: {away_games}")
    
    return team_stats_df

# Usage
if __name__ == "__main__":
    team_stats_file = "team_stats_cleaned.csv"
    games_mapping_file = "nba_games_with_team_mapping.csv"
    output_file = "team_stats_with_home_away.csv"
    
    result_df = add_home_away_info(team_stats_file, games_mapping_file, output_file)
    
    # Display sample of new columns
    print("\nSample of new columns:")
    print(result_df[['GAME_ID', 'TEAM_ID', 'IS_HOME', 'OPPONENT_ID', 'STAT_ID']].head(10))


Updated team stats saved to team_stats_with_home_away.csv
Home games: 1191, Away games: 1191

Sample of new columns:
    GAME_ID     TEAM_ID  IS_HOME OPPONENT_ID             STAT_ID
0  22401160  1610612748    False  1610612741  224011601610612748
1  22401160  1610612741     True  1610612748  224011601610612741
2  22401162  1610612762     True  1610612757  224011621610612762
3  22401162  1610612757    False  1610612762  224011621610612757
4  22401164  1610612756     True  1610612760  224011641610612756
5  22401164  1610612760    False  1610612756  224011641610612760
6  22401158  1610612761     True  1610612766  224011581610612761
7  22401158  1610612766    False  1610612761  224011581610612766
8  22401165  1610612758     True  1610612743  224011651610612758
9  22401165  1610612743    False  1610612758  224011651610612743


In [17]:
import pandas as pd

def add_win_and_scores(team_stats_path, games_path, output_team_stats_path, output_games_path):
    """
    Add WIN field to team stats and HOME_SCORE/AWAY_SCORE fields to games file
    """
    
    # Read both CSV files
    team_stats_df = pd.read_csv(team_stats_path)
    games_df = pd.read_csv(games_path)
    
    # Add WIN field to team stats based on PLUS_MINUS
    team_stats_df['WIN'] = team_stats_df['PLUS_MINUS'] > 0
    
    # Initialize score columns in games file
    games_df['HOME_SCORE'] = 0
    games_df['AWAY_SCORE'] = 0
    
    # Create a lookup dictionary from team stats for scores
    # Key: GAME_ID, Value: {'home_score': pts, 'away_score': pts}
    score_lookup = {}
    
    for _, row in team_stats_df.iterrows():
        game_id = row['GAME_ID']
        pts = row['PTS']
        is_home = row['IS_HOME']
        
        if game_id not in score_lookup:
            score_lookup[game_id] = {'home_score': None, 'away_score': None}
        
        if is_home:
            score_lookup[game_id]['home_score'] = pts
        else:
            score_lookup[game_id]['away_score'] = pts
    
    # Update games file with scores
    for idx, row in games_df.iterrows():
        game_id = row['GAME_ID']
        if game_id in score_lookup:
            if score_lookup[game_id]['home_score'] is not None:
                games_df.at[idx, 'HOME_SCORE'] = score_lookup[game_id]['home_score']
            if score_lookup[game_id]['away_score'] is not None:
                games_df.at[idx, 'AWAY_SCORE'] = score_lookup[game_id]['away_score']
    
    # Save the updated files
    team_stats_df.to_csv(output_team_stats_path, index=False)
    games_df.to_csv(output_games_path, index=False)
    
    print(f"Updated team stats saved to: {output_team_stats_path}")
    print(f"Updated games file saved to: {output_games_path}")
    
    # Summary
    wins = team_stats_df['WIN'].sum()
    total_games = len(team_stats_df)
    print(f"\nSummary:")
    print(f"Total team entries: {total_games}")
    print(f"Wins: {wins}, Losses: {total_games - wins}")
    print(f"Games with complete scores: {len(games_df[(games_df['HOME_SCORE'] > 0) & (games_df['AWAY_SCORE'] > 0)])}")

# Usage
if __name__ == "__main__":
    team_stats_file = "team_stats_with_home_away.csv"  # Your current team stats file
    games_file = "nba_games_with_team_mapping.csv"  # Your games file
    
    output_team_stats = "team_stats_winloss.csv"
    output_games = "games_with_scores.csv"
    
    add_win_and_scores(team_stats_file, games_file, output_team_stats, output_games)


Updated team stats saved to: team_stats_winloss.csv
Updated games file saved to: games_with_scores.csv

Summary:
Total team entries: 2382
Wins: 1191, Losses: 1191
Games with complete scores: 1191


In [14]:
import pandas as pd

# Read the files
games_df = pd.read_csv('games_with_scores.csv')
team_stats_df = pd.read_csv('team_stats_final.csv')

# For games_with_scores: keep only specified columns in specified order
games_columns = [
    'GAME_ID', 'GAME_DATE', 'HOME_TEAM_ID', 'AWAY_TEAM_ID', 'HOME_SCORE', 'AWAY_SCORE'
]
games_filtered = games_df[games_columns]

# For team_stats_final: reorder columns with specified ones first
priority_columns = ['STAT_ID', 'GAME_ID', 'TEAM_ID', 'IS_HOME', 'OPPONENT_ID']
remaining_columns = [col for col in team_stats_df.columns if col not in priority_columns]
new_column_order = priority_columns + remaining_columns
team_stats_reordered = team_stats_df[new_column_order]

# Save the updated files
games_filtered.to_csv('games_final.csv', index=False)
team_stats_reordered.to_csv('team_stats_final.csv', index=False)

print("Files updated successfully!")
print(f"Games file now has {len(games_filtered.columns)} columns")
print(f"Team stats file columns reordered with {len(team_stats_reordered.columns)} total columns")


Files updated successfully!
Games file now has 6 columns
Team stats file columns reordered with 29 total columns


In [24]:
team_stats_df1 = pd.read_csv('team_stats_final.csv')
team_stats_df1.drop(columns=['TEAM_NAME', 'TEAM_ABBREVIATION', 'TEAM_CITY'], inplace=True)
team_stats_df1.to_csv('team_stats_final.csv', index=False)
print("Files updated successfully!")

Files updated successfully!


In [20]:
team_stats_df2 = pd.read_csv('team_stats_final.csv')
team_stats_df2.head()

,STAT_ID,GAME_ID,TEAM_ID,IS_HOME,OPPONENT_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CITY,MIN,FGM,...,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS,WIN
0,224011601610612748,22401160,1610612748,False,1610612741,Heat,MIA,Miami,240:00,44,...,28,39,28,11,3,13,10,111,-8.0,False
1,224011601610612741,22401160,1610612741,True,1610612748,Bulls,CHI,Chicago,240:00,46,...,36,46,33,8,7,16,14,119,8.0,True
2,224011621610612762,22401162,1610612762,True,1610612757,Jazz,UTA,Utah,265:00,50,...,40,56,27,14,4,24,26,133,7.0,True
3,224011621610612757,22401162,1610612757,False,1610612762,Trail Blazers,POR,Portland,265:00,48,...,29,43,27,13,5,16,23,126,-7.0,False
4,224011641610612756,22401164,1610612756,True,1610612760,Suns,PHX,Phoenix,240:00,42,...,29,38,31,10,3,13,17,112,-13.0,False


In [21]:
import pandas as pd

# Read the games file
games_df = pd.read_csv('games_final.csv')

# Remove rows where HOME_SCORE is 0
games_filtered = games_df[games_df['HOME_SCORE'] != 0]

# Save the updated file
games_filtered.to_csv('games_final.csv', index=False)

print(f"Removed {len(games_df) - len(games_filtered)} rows where HOME_SCORE was 0")
print(f"Games file now has {len(games_filtered)} rows")

Removed 39 rows where HOME_SCORE was 0
Games file now has 1191 rows


In [25]:
import pandas as pd

# Read the games file
games_df = pd.read_csv('games_final.csv')

games_df.head()

,GAME_ID,GAME_DATE,HOME_TEAM_ID,AWAY_TEAM_ID,HOME_SCORE,AWAY_SCORE
0,22401191.0,2025-04-13,1610612755,1610612741,0,0
1,22401195.0,2025-04-13,1610612750,1610612762,0,0
2,22401198.0,2025-04-13,1610612744,1610612746,0,0
3,22401200.0,2025-04-13,1610612758,1610612756,0,0
4,22401187.0,2025-04-13,1610612738,1610612766,0,0


In [38]:
import numpy as np
games_df['GAME_ID'] = games_df['GAME_ID'].astype(np.int64)
games_df.to_csv('games_final.csv', index=False)


In [39]:
print(games_df.dtypes)

GAME_ID          int64
GAME_DATE       object
HOME_TEAM_ID     int64
AWAY_TEAM_ID     int64
HOME_SCORE       int64
AWAY_SCORE       int64
dtype: object


In [26]:
import pandas as pd

# Read the games file
stats_df = pd.read_csv('team_stats_final.csv')

stats_df.head()


,STAT_ID,GAME_ID,TEAM_ID,IS_HOME,OPPONENT_ID,MIN,FGM,FGA,FG_PCT,FG3M,...,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS,WIN
0,224011601610612748,22401160,1610612748,False,1610612741,240:00,44,92,0.478,12,...,28,39,28,11,3,13,10,111,-8.0,False
1,224011601610612741,22401160,1610612741,True,1610612748,240:00,46,89,0.517,14,...,36,46,33,8,7,16,14,119,8.0,True
2,224011621610612762,22401162,1610612762,True,1610612757,265:00,50,98,0.510,15,...,40,56,27,14,4,24,26,133,7.0,True
3,224011621610612757,22401162,1610612757,False,1610612762,265:00,48,104,0.462,11,...,29,43,27,13,5,16,23,126,-7.0,False
4,224011641610612756,22401164,1610612756,True,1610612760,240:00,42,87,0.483,15,...,29,38,31,10,3,13,17,112,-13.0,False


In [4]:
print(stats_df.dtypes)

STAT_ID                int64
GAME_ID                int64
TEAM_ID                int64
IS_HOME                 bool
OPPONENT_ID            int64
TEAM_NAME             object
TEAM_ABBREVIATION     object
TEAM_CITY             object
MIN                   object
FGM                    int64
FGA                    int64
FG_PCT               float64
FG3M                   int64
FG3A                   int64
FG3_PCT              float64
FTM                    int64
FTA                    int64
FT_PCT               float64
OREB                   int64
DREB                   int64
REB                    int64
AST                    int64
STL                    int64
BLK                    int64
TO                     int64
PF                     int64
PTS                    int64
PLUS_MINUS           float64
WIN                     bool
dtype: object


In [11]:
pip install pyodbc>=4.0.0 sqlalchemy>=1.4.0

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import pandas as pd
import pyodbc
from sqlalchemy import create_engine
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class CSVToMSSQL:
    def __init__(self, server, database, username=None, password=None, trusted_connection=True):
        """
        Initialize connection to MS SQL Server
        
        Args:
            server: SQL Server instance (e.g., 'localhost' or 'server\\instance')
            database: Database name
            username: SQL Server username (if not using Windows auth)
            password: SQL Server password (if not using Windows auth)
            trusted_connection: Use Windows authentication (default: True)
        """
        self.server = server
        self.database = database
        
        if trusted_connection:
            # Windows Authentication
            connection_string = f"mssql+pyodbc://@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes"
        else:
            # SQL Server Authentication
            connection_string = f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server"
        
        self.engine = create_engine(connection_string)
        logger.info(f"Connected to SQL Server: {server}/{database}")
    
    def create_tables(self):
        """Create the NBA tables in SQL Server"""
        sql_commands = [
            """
            IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='NBA_Teams' AND xtype='U')
            CREATE TABLE NBA_Teams (
                team_id INT PRIMARY KEY,
                team_name NVARCHAR(100) NOT NULL,
                team_abbreviation NVARCHAR(100) NOT NULL,
                conference NVARCHAR(50) NOT NULL,
                division NVARCHAR(50) NOT NULL,
            )
            """,
            """
            IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='games_final' AND xtype='U')
            CREATE TABLE nba_games (
                game_id INT PRIMARY KEY NOT NULL,
                game_date NVARCHAR(50) NOT NULL,
                home_team_id INT NOT NULL,
                away_team_id INT NOT NULL,
                home_score INT,
                away_score INT,
                FOREIGN KEY (home_team_id) REFERENCES NBA_Teams(team_id),
                FOREIGN KEY (away_team_id) REFERENCES NBA_Teams(team_id)
            )
            """,
            """
            IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='team_stats' AND xtype='U')
            CREATE TABLE team_stats (
                Stat_ID INT PRIMARY KEY,
            	Game_ID INT,
            	Team_ID INT,
            	Is_Home BIT,
            	Opponent_ID INT,
            	"MIN" TIME,
            	FGM INT, 
            	FGA INT,
            	FG_Pct DECIMAL,
            	FG3M INT,
            	FG3A INT,
            	FG3_Pct DECIMAL,
            	FTM INT,
            	FTA INT,
            	FT_Pct DECIMAL,
            	OREB INT,
            	DREB INT,
            	REB INT,
            	AST INT,
            	STL INT,
            	BLK INT,
            	"TO" INT,
            	PF INT,
            	PTS INT,
            	PLUS_MINUS DECIMAL,
            	WIN BIT
                FOREIGN KEY (Team_ID) REFERENCES NBA_Teams(team_id),
                FOREIGN KEY (Game_ID) REFERENCES nba_games(game_id),
            )
            """
        ]
        
        with self.engine.connect() as conn:
            for sql in sql_commands:
                conn.execute(sql)
                conn.commit()
        
        logger.info("Tables created successfully")
    
    def load_csv_to_table(self, csv_file, table_name, if_exists='replace'):
        """
        Load CSV file to SQL Server table
        
        Args:
            csv_file: Path to CSV file
            table_name: Name of the target table
            if_exists: What to do if table exists ('replace', 'append', 'fail')
        """
        try:
            # Read CSV file
            df = pd.read_csv(csv_file)
            logger.info(f"Read {len(df)} rows from {csv_file}")
            
            # Load to SQL Server
            df.to_sql(table_name, self.engine, if_exists=if_exists, index=False, method='multi')
            logger.info(f"Successfully loaded {len(df)} rows to {table_name}")
            
        except Exception as e:
            logger.error(f"Error loading {csv_file} to {table_name}: {e}")
            raise
    
    def run_pipeline(self):
        """Run the complete pipeline"""

            
            # Define CSV files and their corresponding tables
        csv_mappings = [
            ('nba_teams.csv', 'NBA_Teams'),
            ('games_final.csv', 'nba_games'),
            ('team_stats_final.csv', 'team_stats')
        ]
            
        # Load each CSV file
        for csv_file, table_name in csv_mappings:
            self.load_csv_to_table(csv_file, table_name)
            
        logger.info("Pipeline completed successfully!")
            
        # Show record counts
        with self.engine.connect() as conn:
            for _, table_name in csv_mappings:
                result = conn.execute(f"SELECT COUNT(*) FROM {table_name}")
                count = result.fetchone()[0]
                logger.info(f"{table_name}: {count} records")
                    
    # except Exception as e:
    #     logger.error(f"Pipeline failed: {e}")
    #     raise

def main():
    """
    Main function - Update these connection details for your SQL Server
    """
    
    # UPDATE THESE CONNECTION DETAILS
    SERVER = "FARZAN-LEGIONS7\SQLEXPRESS"  # or "server\\instance" or IP address
    DATABASE = "NBA_Databse"  # Your database name
    
    # Option 1: Windows Authentication (recommended)
    pipeline = CSVToMSSQL(
        server=SERVER,
        database=DATABASE,
        trusted_connection=True
    )
    
    # Run the pipeline
    pipeline.run_pipeline()

if __name__ == "__main__":
    main()